In [ ]:
import os
import csv
from collections import defaultdict

# Update these paths with your dataset structure
DATA_SPLITS = {
    'train': '/content/drive/MyDrive/extracted_zip/military_object_dataset/train/labels',
    'val': '/content/drive/MyDrive/extracted_zip/military_object_dataset/val/labels',
    'test': '/content/drive/MyDrive/extracted_zip/military_object_dataset/test/labels'
}

output_csv = '/content/drive/MyDrive/extracted_zip/military_object_dataset/class_distribution.csv'
class_split_counts = defaultdict(lambda: {'train': 0, 'val': 0, 'test': 0})

for split_name, label_dir in DATA_SPLITS.items():
    for file in os.listdir(label_dir):
        if file.endswith('.txt'):
            with open(os.path.join(label_dir, file), 'r') as f:
                for line in f:
                    try:
                        class_id = int(float(line.strip().split()[0]))
                        class_split_counts[class_id][split_name] += 1
                    except:
                        continue

# Write CSV in pivot format
all_splits = ['train', 'val', 'test']
with open(output_csv, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['class_id'] + all_splits)
    for class_id in sorted(class_split_counts.keys()):
        row = [class_id] + [class_split_counts[class_id].get(split, 0) for split in all_splits]
        writer.writerow(row)

print(f"Class-wise distribution saved to: {output_csv}")


Class-wise distribution saved to: /content/drive/MyDrive/extracted_zip/military_object_dataset/class_distribution.csv


In [ ]:
import os
import shutil

ROOT_DIR = '/content/drive/MyDrive/extracted_zip/military_object_dataset'
SPLITS = ['train', 'val', 'test']
TARGET_MOVES = {
    11: [('val', 50), ('test', 50)],
    1:  [('test', 50)],
    7:  [('val', 8), ('test', 8)],
}
CLASSES_TO_REMOVE = [5, 9]

def get_all_label_paths(split):
    label_dir = os.path.join(ROOT_DIR, split, 'labels')
    return [os.path.join(label_dir, f) for f in os.listdir(label_dir) if f.endswith('.txt')]

def remove_classes_and_clean(class_ids):
    removed_count = 0
    for split in SPLITS:
        for label_path in get_all_label_paths(split):
            with open(label_path, 'r') as f:
                lines = f.readlines()
            new_lines = [line for line in lines if int(line.strip().split()[0]) not in class_ids]
            if not new_lines:
                # Remove both label and image
                os.remove(label_path)
                image_path = label_path.replace('/labels/', '/images/').replace('.txt', '.jpg')
                if os.path.exists(image_path):
                    os.remove(image_path)
                removed_count += 1
            else:
                with open(label_path, 'w') as f:
                    f.writelines(new_lines)
    print(f"Removed {removed_count} image-label pairs containing only classes {class_ids}")

def get_files_by_class(split, class_id):
    found = []
    for label_path in get_all_label_paths(split):
        with open(label_path, 'r') as f:
            lines = f.readlines()
        if any(int(line.strip().split()[0]) == class_id for line in lines):
            found.append(label_path)
    return found

def move_files(label_paths, dest_split):
    for label_path in label_paths:
        # Paths
        src_img = label_path.replace('/labels/', '/images/').replace('.txt', '.jpg')
        dst_lbl = label_path.replace('/train/', f'/{dest_split}/')
        dst_img = src_img.replace('/train/', f'/{dest_split}/')
        # Move
        os.makedirs(os.path.dirname(dst_lbl), exist_ok=True)
        os.makedirs(os.path.dirname(dst_img), exist_ok=True)
        shutil.move(label_path, dst_lbl)
        if os.path.exists(src_img):
            shutil.move(src_img, dst_img)

def move_target_classes():
    for cls_id, moves in TARGET_MOVES.items():
        src_files = get_files_by_class('train', cls_id)
        for dest_split, count in moves:
            to_move = src_files[:count]
            move_files(to_move, dest_split)
            src_files = src_files[count:]
            print(f"Moved {len(to_move)} class {cls_id} samples to {dest_split}")

if __name__ == "__main__":
    remove_classes_and_clean(CLASSES_TO_REMOVE)
    move_target_classes()
    print("Dataset update complete.")


Removed 705 image-label pairs containing only classes [5, 9]
Moved 50 class 11 samples to val
Moved 50 class 11 samples to test
Moved 50 class 1 samples to test
Moved 8 class 7 samples to val
Moved 8 class 7 samples to test
Dataset update complete.


In [ ]:
import os
import csv
from collections import defaultdict

# Update these paths with your dataset structure
DATA_SPLITS = {
    'train': '/content/drive/MyDrive/extracted_zip/military_object_dataset/train/labels',
    'val': '/content/drive/MyDrive/extracted_zip/military_object_dataset/val/labels',
    'test': '/content/drive/MyDrive/extracted_zip/military_object_dataset/test/labels'
}

output_csv = '/content/drive/MyDrive/extracted_zip/military_object_dataset/class_distribution_final.csv'
class_split_counts = defaultdict(lambda: {'train': 0, 'val': 0, 'test': 0})

for split_name, label_dir in DATA_SPLITS.items():
    for file in os.listdir(label_dir):
        if file.endswith('.txt'):
            with open(os.path.join(label_dir, file), 'r') as f:
                for line in f:
                    try:
                        class_id = int(float(line.strip().split()[0]))
                        class_split_counts[class_id][split_name] += 1
                    except:
                        continue

# Write CSV in pivot format
all_splits = ['train', 'val', 'test']
with open(output_csv, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['class_id'] + all_splits)
    for class_id in sorted(class_split_counts.keys()):
        row = [class_id] + [class_split_counts[class_id].get(split, 0) for split in all_splits]
        writer.writerow(row)

print(f"Class-wise distribution saved to: {output_csv}")


Class-wise distribution saved to: /content/drive/MyDrive/extracted_zip/military_object_dataset/class_distribution_final.csv


In [ ]:
import os
from pathlib import Path
import shutil

LABEL_DIR = '/content/drive/MyDrive/extracted_zip/military_object_dataset/train/labels'
IMAGE_DIR = '/content/drive/MyDrive/extracted_zip/military_object_dataset/train/images'
OUTPUT_DIR = '/content/drive/MyDrive/extracted_zip/augmented_dataset'

aug_target_classes = {7, 8}  # Soldier and civilian vehicle
files_to_augment = []

for label_file in os.listdir(LABEL_DIR):
    path = os.path.join(LABEL_DIR, label_file)
    with open(path, 'r') as f:
        lines = f.readlines()
    for line in lines:
        class_id = int(float(line.strip().split()[0]))  # robust parsing
        if class_id in aug_target_classes:
            files_to_augment.append(label_file)
            break  # Only need one matching class to include this file

print(f"\nTotal files to augment: {len(files_to_augment)}")



Total files to augment: 418


In [ ]:
import cv2
import os
import glob
import albumentations as A

# ✅ Use Pascal VOC (absolute format)
augment = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.Rotate(limit=15, p=0.5),
    A.Blur(blur_limit=3, p=0.2)
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels'], min_visibility=0.3))

def load_yolo_labels(path):
    with open(path, 'r') as f:
        boxes = []
        classes = []
        for line in f:
            c, x, y, w, h = map(float, line.strip().split())
            boxes.append([x, y, w, h])
            classes.append(int(c))
        return boxes, classes

def save_yolo_labels(path, boxes, classes):
    with open(path, 'w') as f:
        for cls, box in zip(classes, boxes):
            f.write(f"{cls} {' '.join(map(str, box))}\n")

def clip_yolo_boxes(boxes):
    """
    Clip YOLO-format boxes (x_center, y_center, w, h) to be within [0, 1].
    """
    clipped = []
    for x, y, w, h in boxes:
        x1 = max(0, x - w / 2)
        y1 = max(0, y - h / 2)
        x2 = min(1, x + w / 2)
        y2 = min(1, y + h / 2)

        new_x = (x1 + x2) / 2
        new_y = (y1 + y2) / 2
        new_w = x2 - x1
        new_h = y2 - y1

        if new_w > 0 and new_h > 0:
            clipped.append([new_x, new_y, new_w, new_h])
    return clipped

# Setup
num_augmentations = 1
aug_img_dir = os.path.join(OUTPUT_DIR, 'images')
aug_lbl_dir = os.path.join(OUTPUT_DIR, 'labels')

# Clean previous output
for f in glob.glob(os.path.join(aug_img_dir, '*')):
    os.remove(f)
for f in glob.glob(os.path.join(aug_lbl_dir, '*')):
    os.remove(f)

print("Cleared existing augmented images and labels.")

# Main loop
for label_file in files_to_augment:
    img_file = label_file.replace('.txt', '.jpg')
    image_path = os.path.join(IMAGE_DIR, img_file)
    label_path = os.path.join(LABEL_DIR, label_file)

    image = cv2.imread(image_path)
    if image is None:
        print(f"Skipping missing image: {image_path}")
        continue

    h, w = image.shape[:2]

    for i in range(num_augmentations):
        boxes, classes = load_yolo_labels(label_path)
        boxes = clip_yolo_boxes(boxes)

        if not boxes:
            continue

        # Convert YOLO → absolute pixel coordinates (Pascal VOC format)
        abs_boxes = []
        for x, y, bw, bh in boxes:
            x_min = (x - bw / 2) * w
            y_min = (y - bh / 2) * h
            x_max = (x + bw / 2) * w
            y_max = (y + bh / 2) * h
            abs_boxes.append([x_min, y_min, x_max, y_max])

        # Apply augmentations
        try:
            augmented = augment(image=image, bboxes=abs_boxes, class_labels=classes)
        except Exception as e:
            print(f"Skipping {image_path} due to augmentation error: {e}")
            continue

        aug_image = augmented['image']
        aug_abs_boxes = augmented['bboxes']
        aug_classes = augmented['class_labels']

        # Convert Pascal VOC → YOLO format and normalize
        aug_yolo_boxes = []
        for x_min, y_min, x_max, y_max in aug_abs_boxes:
            x_min = max(0, min(w, x_min))
            y_min = max(0, min(h, y_min))
            x_max = max(0, min(w, x_max))
            y_max = max(0, min(h, y_max))

            bw = x_max - x_min
            bh = y_max - y_min
            x_center = x_min + bw / 2
            y_center = y_min + bh / 2

            x_center /= w
            y_center /= h
            bw /= w
            bh /= h

            if bw > 0 and bh > 0:
                aug_yolo_boxes.append([x_center, y_center, bw, bh])

        if aug_yolo_boxes:
            aug_img_name = f'aug_{i}_{img_file}'
            aug_lbl_name = f'aug_{i}_{label_file}'

            cv2.imwrite(os.path.join(aug_img_dir, aug_img_name), aug_image)
            save_yolo_labels(os.path.join(aug_lbl_dir, aug_lbl_name), aug_yolo_boxes, aug_classes)


Cleared existing augmented images and labels.
Skipping /content/drive/MyDrive/extracted_zip/military_object_dataset/train/images/009464.jpg due to augmentation error: The lengths of bboxes and class_labels do not match. Got 2 and 3 respectively.
Skipping /content/drive/MyDrive/extracted_zip/military_object_dataset/train/images/009590.jpg due to augmentation error: The lengths of bboxes and class_labels do not match. Got 5 and 6 respectively.
Skipping /content/drive/MyDrive/extracted_zip/military_object_dataset/train/images/009621.jpg due to augmentation error: The lengths of bboxes and class_labels do not match. Got 7 and 8 respectively.
Skipping /content/drive/MyDrive/extracted_zip/military_object_dataset/train/images/010042.jpg due to augmentation error: The lengths of bboxes and class_labels do not match. Got 6 and 7 respectively.
Skipping /content/drive/MyDrive/extracted_zip/military_object_dataset/train/images/010114.jpg due to augmentation error: The lengths of bboxes and class_l

In [ ]:
import os

LABEL_DIR = '/content/drive/MyDrive/extracted_zip/augmented_dataset/labels'

class_counts = {7: 0, 8: 0}  # civilian and trench

for label_file in os.listdir(LABEL_DIR):
    if label_file.endswith('.txt'):
        path = os.path.join(LABEL_DIR, label_file)
        with open(path, 'r') as f:
            lines = f.readlines()
        for line in lines:
            class_id = int(float(line.split()[0]))
            if class_id in class_counts:
                class_counts[class_id] += 1

print(f"Class 7 (civilian): {class_counts[7]} instances")
print(f"Class 8 (trench): {class_counts[8]} instances")

Class 7 (civilian): 452 instances
Class 8 (trench): 424 instances


In [ ]:
import os
import csv
from collections import defaultdict

# Update these paths with your dataset structure
DATA_SPLITS = {
    'train': '/content/drive/MyDrive/extracted_zip/military_object_dataset/train/labels',
    'val': '/content/drive/MyDrive/extracted_zip/military_object_dataset/val/labels',
    'test': '/content/drive/MyDrive/extracted_zip/military_object_dataset/test/labels'
}

output_csv = '/content/drive/MyDrive/extracted_zip/military_object_dataset/class_distribution_withaug_final.csv'
class_split_counts = defaultdict(lambda: {'train': 0, 'val': 0, 'test': 0})

for split_name, label_dir in DATA_SPLITS.items():
    for file in os.listdir(label_dir):
        if file.endswith('.txt'):
            with open(os.path.join(label_dir, file), 'r') as f:
                for line in f:
                    try:
                        class_id = int(float(line.strip().split()[0]))
                        class_split_counts[class_id][split_name] += 1
                    except:
                        continue

# Write CSV in pivot format
all_splits = ['train', 'val', 'test']
with open(output_csv, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['class_id'] + all_splits)
    for class_id in sorted(class_split_counts.keys()):
        row = [class_id] + [class_split_counts[class_id].get(split, 0) for split in all_splits]
        writer.writerow(row)

print(f"Class-wise distribution saved to: {output_csv}")


Class-wise distribution saved to: /content/drive/MyDrive/extracted_zip/military_object_dataset/class_distribution_withaug_final.csv


In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [6]:
# Copy the zip from Drive to local fast storage
!cp /content/drive/MyDrive/copied_file.zip /content/

# Unzip it locally
!unzip /content/copied_file.zip -d /content/military_OD_data/

Streaming output truncated to the last 5000 lines.
  inflating: /content/military_OD_data/military_object_dataset/val/images/011492.jpg  
  inflating: /content/military_OD_data/military_object_dataset/val/images/011493.jpg  
  inflating: /content/military_OD_data/military_object_dataset/val/images/011494.jpg  
  inflating: /content/military_OD_data/military_object_dataset/val/images/011495.jpg  
  inflating: /content/military_OD_data/military_object_dataset/val/images/011496.jpg  
  inflating: /content/military_OD_data/military_object_dataset/val/images/011497.jpg  
  inflating: /content/military_OD_data/military_object_dataset/val/images/011498.jpg  
  inflating: /content/military_OD_data/military_object_dataset/val/images/011499.jpg  
  inflating: /content/military_OD_data/military_object_dataset/val/images/011500.jpg  
  inflating: /content/military_OD_data/military_object_dataset/val/images/011501.jpg  
  inflating: /content/military_OD_data/military_object_dataset/val/images/01150

In [ ]:
import os
import shutil
from ultralytics import YOLO

# === STEP 1: Set paths ===
# Google Drive paths (permanent storage)
drive_dataset_path = '/content/drive/MyDrive/extracted_zip/military_object_dataset'
drive_training_dir = '/content/drive/MyDrive/yolo_training/military_detection/yolov8_military'  # Saves here
drive_results_dir = os.path.join(drive_training_dir, 'predict_results')

# Local paths (fast access)
local_dataset_path = '/content/military_OD_data/military_object_dataset'
yaml_file_path = os.path.join(local_dataset_path, 'military_dataset.yaml')

# === STEP 3: Load pre-trained model (or resume from last checkpoint) ===
last_ckpt_path = os.path.join(drive_training_dir, 'weights', 'last.pt')
if os.path.exists(last_ckpt_path):
    print("🔁 Resuming from last checkpoint")
    model = YOLO(last_ckpt_path)
else:
    print("🆕 Starting fresh training")
    model = YOLO('yolov8n.pt')  # or yolov8s.pt, yolov8m.pt etc.

# === STEP 4: Train ===
model.train(
    data=yaml_file_path,
    epochs=50,
    imgsz=640,
    batch=16,
    patience=10,
    project='yolo_training',
    name='military_detection/yolov8_military',  # Folder path inside /content/drive/MyDrive/yolo_training/
    exist_ok=True,  # Don't raise error if folder exists
    verbose=True,
    resume=os.path.exists(last_ckpt_path)  # Ensures resume flag is set correctly
)

# === STEP 5: Validate (optional) ===
metrics = model.val()
print(metrics)

# === STEP 6: Predict on test set and save to Drive ===
results = model.predict(
    source=os.path.join(local_dataset_path, 'test/images'),
    save=True,
    save_txt=True,
    project='yolo_training',
    name='military_detection/yolov8_military/predict_results'
)

# === STEP 7: Export model to ONNX format (optional) ===
model.export(format='onnx')


🆕 Starting fresh training
Ultralytics 8.3.133 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/military_OD_data/military_object_dataset/military_dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8_military, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overl

train: Scanning /content/military_OD_data/military_object_dataset/train/labels... 21978 images, 294 backgrounds, 0 corrupt: 100%|██████████| 21978/21978 [00:28<00:00, 773.63it/s] 


train: New cache created: /content/military_OD_data/military_object_dataset/train/labels.cache
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 22, len(boxes) = 43336. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 679.4±593.0 MB/s, size: 113.2 KB)


val: Scanning /content/military_OD_data/military_object_dataset/val/labels... 2941 images, 273 backgrounds, 0 corrupt: 100%|██████████| 2941/2941 [00:02<00:00, 1373.25it/s]

val: New cache created: /content/military_OD_data/military_object_dataset/val/labels.cache


...

      10/50      3.14G      1.296       1.38      1.385         42        640: 100%|██████████| 1374/1374 [07:38<00:00,  2.99it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 92/92 [00:35<00:00,  2.58it/s]


                   all       2941       5081      0.486      0.428      0.407      0.248

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      11/50      3.16G      1.274      1.346      1.371         32        640: 100%|██████████| 1374/1374 [07:42<00:00,  2.97it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 92/92 [00:34<00:00,  2.65it/s]


                   all       2941       5081      0.615      0.385      0.425      0.263

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      12/50      3.16G      1.266      1.325      1.367         78        640:  65%|██████▌   | 898/1374 [05:00<02:47,  2.85it/s]